## Setup experiment

### Import dependencies

In [1]:
import torch
from tqdm import tqdm
import pandas as pd

tqdm.pandas()

from transformers import pipeline, AutoTokenizer
from datasets import load_dataset

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/mymusise/pro/stable-diffusion-webui/venv/lib/python3.8/site-packages/bitsandbytes-0.39.0-py3.8.egg/bitsandbytes/libbitsandbytes_cuda117.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/mymusise/pro/stable-diffusion-webui/venv/lib/python3.8/site-packages/bitsandbytes-0.39.0-py3.8.egg/bitsandbytes/libbitsandbytes_cuda117.so...


/home/mymusise/pro/stable-diffusion-webui/venv/lib/python3.8/site-packages/bitsandbytes-0.39.0-py3.8.egg/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/home/mymusise/pro/stable-diffusion-webui/venv/lib/python3.8/site-packages/bitsandbytes-0.39.0-py3.8.egg/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('"zh-cn","availableLanguages"'), PosixPath('"zh-cn"},"_languagePackId"'), PosixPath('{"locale"'), PosixPath('"/home/mymusise/.vscode-server/data/clp/fa871dcc52f689d5c3b8a42d026c7d06.zh-cn/corrupted.info","_languagePackSupport"'), PosixPath('"/home/mymusise/.vscode-server/data/clp/fa871dcc52f689d5c3b8a42d026c7d06.zh-cn/tcf.json","_cacheRoot"'), PosixPath('{"*"'), PosixPath('"/home/mymusise/.vscode-server/data/clp/fa871dcc52f689d5c3b8a

### Configuration

In [2]:
config = PPOConfig(
    model_name="lvwerra/gpt2-imdb",
    learning_rate=1.41e-5,
    log_with="wandb",
)

sent_kwargs = {"return_all_scores": True, "function_to_apply": "none", "batch_size": 16}

In [ ]:
import wandb

wandb.init()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mymusise. Use `wandb login --relogin` to force relogin


: 

: 

You can see that we load a GPT2 model called `gpt2_imdb`. This model was additionally fine-tuned on the IMDB dataset for 1 epoch with the huggingface [script](https://github.com/huggingface/transformers/blob/master/examples/run_language_modeling.py) (no special settings). The other parameters are mostly taken from the original paper ["Fine-Tuning Language Models from Human Preferences"](
https://arxiv.org/pdf/1909.08593.pdf). This model as well as the BERT model is available in the Huggingface model zoo [here](https://huggingface.co/models). The following code should automatically download the models.

## Load data and models

### Load IMDB dataset
The IMDB dataset contains 50k movie review annotated with "positive"/"negative" feedback indicating the sentiment.  We load the IMDB dataset into a DataFrame and filter for comments that are at least 200 characters. Then we tokenize each text and cut it to random size with the `LengthSampler`.

In [ ]:
def build_dataset(config, dataset_name="imdb", input_min_text_length=2, input_max_text_length=8):
    """
    Build dataset for training. This builds the dataset from `load_dataset`, one should
    customize this function to train the model on its own dataset.

    Args:
        dataset_name (`str`):
            The name of the dataset to be loaded.

    Returns:
        dataloader (`torch.utils.data.DataLoader`):
            The dataloader for the dataset.
    """
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    tokenizer.pad_token = tokenizer.eos_token
    # load imdb with datasets
    ds = load_dataset(dataset_name, split="train")
    ds = ds.rename_columns({"text": "review"})
    ds = ds.filter(lambda x: len(x["review"]) > 200, batched=False)

    input_size = LengthSampler(input_min_text_length, input_max_text_length)

    def tokenize(sample):
        sample["input_ids"] = tokenizer.encode(sample["review"])[: input_size()]
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch")
    return ds

: 

: 

In [ ]:
dataset = build_dataset(config)

def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

Found cached dataset imdb (/home/mymusise/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)
Loading cached processed dataset at /home/mymusise/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-b13b5100b4cec2ba.arrow
Loading cached processed dataset at /home/mymusise/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-2fdb6b76d3a0aeb8.arrow


: 

: 

### Load pre-trained GPT2 language models

We load the GPT2 model with a value head and the tokenizer. We load the model twice; the first model is optimized while the second model serves as a reference to calculate the KL-divergence from the starting point. This serves as an additional reward signal in the PPO training to make sure the optimized model does not deviate too much from the original language model.

In [ ]:
model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
tokenizer = AutoTokenizer.from_pretrained(config.model_name)

tokenizer.pad_token = tokenizer.eos_token

: 

: 

### Initialize PPOTrainer
The `PPOTrainer` takes care of device placement and optimization later on:

In [ ]:
ppo_trainer = PPOTrainer(config, model, ref_model, tokenizer, dataset=dataset, data_collator=collator)

: 

: 

### Load BERT classifier
We load a BERT classifier fine-tuned on the IMDB dataset.

In [ ]:
device = ppo_trainer.accelerator.device
if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if torch.cuda.is_available() else "cpu"  # to avoid a `pipeline` bug
sentiment_pipe = pipeline("sentiment-analysis", model="lvwerra/distilbert-imdb", device=device)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


: 

: 

The model outputs are the logits for the negative and positive class. We will use the logits for positive class as a reward signal for the language model.

In [ ]:
text = "this movie was really bad!!"
sentiment_pipe(text, **sent_kwargs)

/home/mymusise/pro/stable-diffusion-webui/venv/lib/python3.8/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[[{'label': 'NEGATIVE', 'score': 2.3350484371185303},
  {'label': 'POSITIVE', 'score': -2.726576566696167}]]

: 

: 

In [ ]:
text = "this movie was really good!!"
sentiment_pipe(text, **sent_kwargs)

[[{'label': 'NEGATIVE', 'score': -2.2947897911071777},
  {'label': 'POSITIVE', 'score': 2.557039737701416}]]

: 

: 

### Generation settings
For the response generation we just use sampling and make sure top-k and nucleus sampling are turned off as well as a minimal length.

In [ ]:
gen_kwargs = {"min_length": -1, "top_k": 0.0, "top_p": 1.0, "do_sample": True, "pad_token_id": tokenizer.eos_token_id}

: 

: 

## Optimize model

### Training loop

The training loop consists of the following main steps:
1. Get the query responses from the policy network (GPT-2)
2. Get sentiments for query/responses from BERT
3. Optimize policy with PPO using the (query, response, reward) triplet

**Training time**

This step takes **~2h** on a V100 GPU with the above specified settings.

In [ ]:
output_min_length = 4
output_max_length = 16
output_length_sampler = LengthSampler(output_min_length, output_max_length)


generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}


for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    query_tensors = batch["input_ids"]

    #### Get response from gpt2
    response_tensors = []
    for query in query_tensors:
        gen_len = output_length_sampler()
        generation_kwargs["max_new_tokens"] = gen_len
        response = ppo_trainer.generate(query, **generation_kwargs)
        response_tensors.append(response.squeeze()[-gen_len:])
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]

    #### Compute sentiment score
    texts = [q + r for q, r in zip(batch["query"], batch["response"])]
    pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
    rewards = [torch.tensor(output[1]["score"]) for output in pipe_outputs]

    #### Run PPO step
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)

0it [00:02, ?it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:23                                                                                   │
│                                                                                                  │
│   20 │   for query in query_tensors:                                                             │
│   21 │   │   gen_len = output_length_sampler()                                                   │
│   22 │   │   generation_kwargs["max_new_tokens"] = gen_len                                       │
│ ❱ 23 │   │   response = ppo_trainer.generate(query, **generation_kwargs)                         │
│   24 │   │   response_tensors.append(response.squeeze()[-gen_len:])                              │
│   25 │   batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]           │
│   26                                                                                             │
│                                                                                                  │
│ /home/mymusise/pro/trl/trl/trainer/ppo_trainer.py:425 in generate                                │
│                                                                                                  │
│    422 │   │   else:                                                                             │
│    423 │   │   │   if length_sampler is not None:                                                │
│    424 │   │   │   │   generation_kwargs["max_new_tokens"] = length_sampler()                    │
│ ❱  425 │   │   │   response = self.accelerator.unwrap_model(self.model).generate(                │
│    426 │   │   │   │   input_ids=query_tensor.unsqueeze(dim=0), **generation_kwargs              │
│    427 │   │   │   )                                                                             │
│    428                                                                                           │
│                                                                                                  │
│ /home/mymusise/pro/trl/trl/models/modeling_value_head.py:195 in generate                         │
│                                                                                                  │
│   192 │   │   │   **kwargs (`dict`, *optional*):                                                 │
│   193 │   │   │   │   Keyword arguments passed to the `generate` method of the wrapped model.    │
│   194 │   │   """                                                                                │
│ ❱ 195 │   │   return self.pretrained_model.generate(*args, **kwargs)                             │
│   196 │                                                                                          │
│   197 │   def state_dict(self, *args, **kwargs):                                                 │
│   198 │   │   r"""                                                                               │
│                                                                                                  │
│ /home/mymusise/pro/stable-diffusion-webui/venv/lib/python3.8/site-packages/torch/autograd/grad_m │
│ ode.py:27 in decorate_context                                                                    │
│                                                                                                  │
│    24 │   │   @functools.wraps(func)                                                             │
│    25 │   │   def decorate_context(*args, **kwargs):                                             │
│    26 │   │   │   with self.clone():                                                             │
│ ❱  27 │   │   │   │   return func(*args, **kwargs)                                               │
│    28 │   │   return cast(F, decorate_context)                                                   │
│    29 │                                                    

: 

: 

### Training progress
If you are tracking the training progress with Weights&Biases you should see a plot similar to the one below. Check out the interactive sample report on wandb.ai: [link](https://app.wandb.ai/lvwerra/trl-showcase/runs/1jtvxb1m/).

<div style="text-align: center">
<img src='https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/gpt2_tuning_progress.png' width='800'>
<p style="text-align: center;"> <b>Figure:</b> Reward mean and distribution evolution during training. </p>
</div>

One can observe how the model starts to generate more positive outputs after a few optimisation steps.

> Note: Investigating the KL-divergence will probably show that at this point the model has not converged to the target KL-divergence, yet. To get there would require longer training or starting with a higher initial coefficient.

## Model inspection
Let's inspect some examples from the IMDB dataset. We can use `model_ref` to compare the tuned model `model` against the model before optimisation.

In [ ]:
#### get a batch from the dataset
bs = 16
game_data = dict()
dataset.set_format("pandas")
df_batch = dataset[:].sample(bs)
game_data["query"] = df_batch["query"].tolist()
query_tensors = df_batch["input_ids"].tolist()

response_tensors_ref, response_tensors = [], []

#### get response from gpt2 and gpt2_ref
for i in range(bs):
    gen_len = output_length_sampler()
    output = ref_model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors_ref.append(output)
    output = model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors.append(output)

#### decode responses
game_data["response (before)"] = [tokenizer.decode(response_tensors_ref[i]) for i in range(bs)]
game_data["response (after)"] = [tokenizer.decode(response_tensors[i]) for i in range(bs)]

#### sentiment analysis of query/response pairs before/after
texts = [q + r for q, r in zip(game_data["query"], game_data["response (before)"])]
game_data["rewards (before)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

texts = [q + r for q, r in zip(game_data["query"], game_data["response (after)"])]
game_data["rewards (after)"] = [output[1]["score"] for output in sentiment_pipe(texts, **sent_kwargs)]

# store results in a dataframe
df_results = pd.DataFrame(game_data)
df_results

/home/mymusise/pro/stable-diffusion-webui/venv/lib/python3.8/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
/home/mymusise/pro/stable-diffusion-webui/venv/lib/python3.8/site-packages/transformers/pipelines/base.py:1079: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


,query,response (before),response (after),rewards (before),rewards (after)
0,Characters you,"will find, wondering what's",know these people. They're,1.312295,1.571384
1,please save,your catalog! -We,... do you want heart,0.406011,-0.039472
2,As a,"wooden production, you can't really",very good musical you can tell all,-2.546566,2.765390
3,Screenwriter Lisa L,oomis will play the main character. It's a rom...,oner obviously began working on this movie as ...,2.136905,0.624396
4,I was sk,"imping on Ruby. I mean, I kept waiting to",ittish and I enjoyed it. The characters were f...,-1.714651,2.667689
5,Carlo Ver,gara's story for Manfall and the team are,de is amazing for most young actress looking t...,1.494800,2.553041
6,Pretty disappointing prequel,". Looks funny, Weight isn't funny and",. I loved it and would recommend it to,-2.915701,0.244960
7,I had,realized Levin at 9 June that DrM's,a very good idea to this film when this,0.474918,0.817296
8,There's no,"heaviness to it, which, despite every expecta...",doubt about that. I absolutely loved this sho...,0.034231,2.744842
9,Columbo movies have,"drags, the script is poor,","been amazing, with how much this one",-2.937638,2.335062


: 

: 

Looking at the reward mean/median of the generated sequences we observe a significant difference.

In [ ]:
print("mean:")
display(df_results[["rewards (before)", "rewards (after)"]].mean())
print()
print("median:")
display(df_results[["rewards (before)", "rewards (after)"]].median())

mean:


rewards (before)   -0.134817
rewards (after)     1.631828
dtype: float64


median:


rewards (before)    0.440465
rewards (after)     1.911272
dtype: float64

: 

: 

## Save model
Finally, we save the model and push it to the Hugging Face for later usage.

In [ ]:
model.save_pretrained("gpt2-imdb-pos-v2")
tokenizer.save_pretrained("gpt2-imdb-pos-v2")

('gpt2-imdb-pos-v2/tokenizer_config.json',
 'gpt2-imdb-pos-v2/special_tokens_map.json',
 'gpt2-imdb-pos-v2/vocab.json',
 'gpt2-imdb-pos-v2/merges.txt',
 'gpt2-imdb-pos-v2/added_tokens.json',
 'gpt2-imdb-pos-v2/tokenizer.json')

: 

: 

: 

: 